In [1]:
import numpy as np
import pandas as pd
import shapefile

from bokeh.plotting import figure, show
from bokeh.layouts import row, column
from bokeh.io import output_notebook    # In-notebook visualisation
output_notebook()

from scipy.spatial import ConvexHull
from scipy.spatial import Delaunay

from scipy.interpolate import splprep, splev
from sklearn.decomposition import PCA

Loading BokehJS ...

In [2]:
shape_path = 'GBR_adm0.shp'
shape_obj  = shapefile.Reader(shape_path)
raw_polys  = np.array(shape_obj.shapes()[0].points)
all_polys  = np.split(raw_polys, shape_obj.shapes()[0].parts)[1:]
poly_num   = len(all_polys)
poly_lengths = [len(poly) for poly in all_polys]
indices      = np.argsort(poly_lengths)
all_polys    = [all_polys[indices[poly_num-i-1]] for i in range(poly_num)]

# Locating bounding boxes

In [3]:
def bbox(array):
    x_max, y_max = np.max(array, axis=0)
    x_min, y_min = np.min(array, axis=0)
    
    return np.array([x_min, x_max, y_min, y_max])


def collision(box_a, box_b):
    
    x_bool = (box_a[0] <= box_b[1]) & (box_b[0] <= box_a[1])
    y_bool = (box_a[2] <= box_b[3]) & (box_b[2] <= box_a[3])
    
    return x_bool & y_bool


def intersect(array, box):
    
    x_bool = (array[:,0] >= box[0]) & (array[:,0] <= box[1])
    y_bool = (array[:,1] >= box[2]) & (array[:,1] <= box[3])
    bools  = x_bool & y_bool
    if bools[0]:
        indx = np.where(~bools)[-1]+1
    else:
        indx = np.where(bools)[0][0]
    
    rolled_bools = np.roll(bools, -indx, axis=0)
    rolled_array = np.roll(array, -indx, axis=0)
    last_indx = np.where(rolled_bools)[0][-1]
#     print(rolled_bools)
#     print(rolled_bools.shape)
#     print(np.where(rolled_bools))
#     print(last_indx)
    
    return rolled_array[:last_indx]

In [4]:
bboxes = [bbox(poly) for poly in all_polys]
orkney_box = np.array([-3.8, -2.0, 58.3, 60.0])
orkney_bools = [collision(orkney_box, box) for box in bboxes]
isles = [j for i, j in zip(orkney_bools, all_polys) if i]
isles[0] = intersect(isles[0], orkney_box)
print(f'Number of isles found: {len(isles)}')

Number of isles found: 54


In [5]:
p = figure(match_aspect=True)

for isle in isles:
    p.line(isle[:,0], isle[:,1], line_width=1.5)
    
show(p)

# Conversion to polar

In [7]:
def latlong_distances(A, B, degrees=True, radius=6371):
    '''Return spherical distance matrix on Earth.
    Input ndarrays of shape (a, 2), (b, 2). Columns : 0=longitude, 1=latitude
    Output ndarray of shape (a, b).
    
    distance = acos(sin(lat1)*sin(lat2)+cos(lat1)*cos(lat2)*cos(lon2-lon1))*6371
    '''
    k = np.pi/180 if degrees else 1
    arr_a = (A*k)[:,None,:]
    arr_b = (B*k)[None,:,:]
    
    sines    = np.sin(arr_a[:,:,1])*np.sin(arr_b[:,:,1])
    cosines  = np.cos(arr_a[:,:,1])*np.cos(arr_b[:,:,1])
    long_cos = np.cos(arr_b[:,:,0] - arr_a[:,:,0])
    argument = sines + cosines*long_cos
    return np.arccos(argument)*radius


def latlong_bearings(A, B, degrees=True):
    '''Return bearings matrix on a sphere.
    Input ndarrays of shape (a, 2), (b, 2). Columns : 0=longitude, 1=latitude
    Output ndarray of shape (a, b).
    
    bearing = arctan2(
        sin(lon2-lon1)*cos(lat2),
        cos(lat1)*sin(lat2) - sin(lat1)*cos(lat2)*cos(lon2-lon1)
    )
    '''
    k = np.pi/180 if degrees else 1
    arr_a = (A*k)[:,None,:]
    arr_b = (B*k)[None,:,:]
    
    arg_1  = np.sin(arr_b[:,:,0] - arr_a[:,:,0])*np.cos(arr_b[:,:,1])
    arg_2a = np.cos(arr_a[:,:,1])*np.sin(arr_b[:,:,1])
    arg_2b = np.sin(arr_a[:,:,1])*np.cos(arr_b[:,:,1])*np.cos(arr_b[:,:,0]-arr_a[:,:,0])
    
    return np.arctan2(arg_1, arg_2a - arg_2b)

In [22]:
kirkwall = np.array([[-2.96, 58.98]])

distances = [latlong_distances(isle, kirkwall) for isle in isles]
bearings  = [latlong_bearings(isle, kirkwall) for isle in isles]

polar_isles = [np.c_[-r*np.sin(a), -r*np.cos(a)] for r,a in zip(distances, bearings)]

In [28]:
paper = np.array([
    [-42, -59.4],
    [-42,  59.4],
    [ 42,  59.4],
    [ 42, -59.4],
    [-42, -59.4]
])
paper += np.array([3, 2])


p = figure(match_aspect=True)

for isle in polar_isles:
    p.line(isle[:,0], isle[:,1], line_width=1.5)
    
p.line(paper[:,0], paper[:,1], line_color='gold', line_width=2)

show(p)